In [1]:
# Libraries
import pandas as pd
import sqlite3 as lite

con = lite.connect('../data/election.db')
cur = con.cursor() 

In [2]:
# Load data
pres2012_xl = pd.ExcelFile('../data/input/presidentielles-2012.xls')
pres2012_dfs = {sheet_name: pres2012_xl.parse(sheet_name) 
          for sheet_name in pres2012_xl.sheet_names}

Check Departements & Communes table

In [44]:
geos = pres2012_dfs['Tour 2'][['Libellé de la commune', 
                               'Code de la commune', 
                               'Libellé du département',
                               'Code du département']]

sql_deps = pd.read_sql_query("SELECT * FROM Departements", con)
sql_deps_lst = sql_deps['DeptCode'].astype(str).tolist()
if not set(geos['Code du département'].drop_duplicates().tolist()).issubset(set(sql_deps_lst)):
    #missing_dpts = set(geos['Code du département'].drop_duplicates().tolist()) - set(sql_deps_lst)
    dpts = geos[['Libellé du département', 'Code du département']].drop_duplicates()
    dpts = dpts[dpts.apply(lambda x: str(x['Code du département']) not in sql_deps_lst, axis = 1)]
    cur.executemany("""
        INSERT INTO Departements(Id, Nom, DeptCode) 
        VALUES(NULL, ?, ?)""", dpts.values.tolist())
    con.commit()
    

sql_deps = pd.read_sql_query("SELECT * FROM Departements", con)
sql_coms = pd.read_sql_query("SELECT C.*, D.DeptCode FROM Communes C LEFT JOIN Departements D on D.Id = C.DeptId", con)
sql_coms_lst = sql_coms[['CommCode', 'DeptCode']].values.tolist()
cmns = geos[['Libellé de la commune', 'Code de la commune','Code du département']].drop_duplicates()
cmns['Code du département'] = cmns['Code du département'].astype(str)
cmns = pd.merge(cmns, sql_coms, how='left', 
             left_on=['Code de la commune','Code du département'],
             right_on=['CommCode', 'DeptCode'])
cmns = cmns[cmns['Id'].isnull()][['Libellé de la commune', 'Code de la commune', 'Code du département']]
cmns = pd.merge(cmns, sql_deps, how='left', 
             left_on=['Code du département'],
             right_on=['DeptCode'])
cmns = cmns[['Libellé de la commune', 'Code de la commune', 'Id']]
if len(cmns.index)>0:
    cur.executemany("""
    INSERT INTO Communes(Id, Nom, CommCode, DeptId) 
    VALUES(NULL, ?, ?, ?)""", cmns.values.tolist())
    con.commit()
    
print([len(dpts.index), len(cmns.index)])
coms_tbl = pd.read_sql_query("SELECT C.Id, C.CommCode, D.DeptCode FROM Communes C LEFT JOIN Departements D on D.Id = C.DeptId", con)

[0, 0]


In [77]:
pres2012_T2 = pres2012_dfs['Tour 2']
o_nrow = len(pres2012_T2.index)
# removing % columns
pres2012_T2 = pres2012_T2.drop([x for x in pres2012_T2.columns.values.tolist() if x.rfind('%')>-1],axis=1)
pres2012_T2 = pres2012_T2.drop(['Votants', 'Exprimés', 
                                'Libellé du département', 'Libellé de la commune'],axis=1)
pres2012_T2['Code du département'] = pres2012_T2['Code du département'].astype(str)
pres2012_T2 = pd.merge(coms_tbl, pres2012_T2, 
                       left_on = ['CommCode', 'DeptCode'], 
                       right_on = ['Code de la commune', 'Code du département'],
                       how = 'right')
pres2012_T2 = pres2012_T2.drop(['Code du département', 'Code de la commune',
                               'CommCode', 'DeptCode'],axis=1)
n_cand = len([x for x in pres2012_T2.columns.values.tolist() if x.rfind('Nom')>-1])

for i in range(0, n_cand):
    if i == 0:
        pres2012_T2.loc[:,'cand.0'] = pres2012_T2[['Sexe','Prénom','Nom']].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T2.loc[:,'Voix.0'] = pres2012_T2['Voix'] 
        pres2012_T2.drop(['Sexe','Prénom','Nom','Voix'],inplace=True,axis=1)
    else:
        i_s = str(i)
        pres2012_T2.loc[:,'cand.'+i_s] = pres2012_T2[['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s]].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T2.drop(['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s],inplace=True,axis=1)

n_nrow = len(pres2012_T2.index)
print([o_nrow, n_nrow])
print(pres2012_T2.isnull().sum())


[36791, 36791]
Id                0
Inscrits          0
Abstentions       0
Blancs et nuls    0
Voix.1            0
cand.0            0
Voix.0            0
cand.1            0
dtype: int64


In [88]:
# Extract candidats names
#pd.melt(pres2012_T2.tail(n = 2), id_vars = ['Id'])
root = pres2012_T2.tail(n = 2)[['Id','Inscrits','Abstentions','Blancs et nuls']]
tmp  = pres2012_T2.tail(n = 2).drop(['Inscrits','Abstentions','Blancs et nuls'],axis=1)
#for i in range(0, n_cand):
#    tmp =
tmp = pd.melt(tmp, id_vars = ['Id'])
tmp = tmp[tmp.apply(lambda x: x['variable'].rfind('cand')>-1, axis = 1)]
tmp.drop('variable', axis = 1)

,Id,value
2,25262,M_François_HOLLANDE
3,25204,M_François_HOLLANDE
6,25262,M_Nicolas_SARKOZY
7,25204,M_Nicolas_SARKOZY


In [59]:
pres2012_T1 = pres2012_dfs['Tour 1']
o_nrow = len(pres2012_T1.index)
# removing % columns
pres2012_T1 = pres2012_T1.drop([x for x in pres2012_T1.columns.values.tolist() if x.rfind('%')>-1],axis=1)
pres2012_T1 = pres2012_T1.drop(['Votants', 'Exprimés', 
                                'Libellé du département', 'Libellé de la commune'],axis=1)
pres2012_T1['Code du département'] = pres2012_T1['Code du département'].astype(str)
pres2012_T1 = pd.merge(coms_tbl, pres2012_T1, 
                       left_on = ['CommCode', 'DeptCode'], 
                       right_on = ['Code de la commune', 'Code du département'],
                       how = 'right')
pres2012_T1 = pres2012_T1.drop(['Code du département', 'Code de la commune',
                               'CommCode', 'DeptCode'],axis=1)
#pres2012_T1 = pres2012_T1.drop(['CommId', 'DeptId',
#                                'Code de la commune', 'Code du département',
#                                'Libellé du département',
#                                'Libellé de la commune'],axis=1)
# Reshaping:
n_cand = len([x for x in pres2012_T1.columns.values.tolist() if x.rfind('Nom')>-1])

for i in range(0, n_cand):
    if i == 0:
        pres2012_T1.loc[:,'cand.0'] = pres2012_T1[['Sexe','Prénom','Nom']].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T1.loc[:,'Voix.0'] = pres2012_T1['Voix'] 
        pres2012_T1.drop(['Sexe','Prénom','Nom','Voix'],inplace=True,axis=1)
    else:
        i_s = str(i)
        pres2012_T1.loc[:,'cand.'+i_s] = pres2012_T1[['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s]].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T1.drop(['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s],inplace=True,axis=1)

n_nrow = len(pres2012_T1.index)
print([o_nrow, n_nrow])
print(pres2012_T1.isnull().sum())
pres2012_T1.tail(n = 5)

[36791, 36791]
Id                0
Inscrits          0
Abstentions       0
Blancs et nuls    0
Voix.1            0
Voix.2            0
Voix.3            0
Voix.4            0
Voix.5            0
Voix.6            0
Voix.7            0
Voix.8            0
Voix.9            0
cand.0            0
Voix.0            0
cand.1            0
cand.2            0
cand.3            0
cand.4            0
cand.5            0
cand.6            0
cand.7            0
cand.8            0
cand.9            0
dtype: int64


,Id,Inscrits,Abstentions,Blancs et nuls,Voix.1,Voix.2,Voix.3,Voix.4,Voix.5,Voix.6,...,Voix.0,cand.1,cand.2,cand.3,cand.4,cand.5,cand.6,cand.7,cand.8,cand.9
36786,25609,621,61,14,101,151,49,6,1,1,...,15,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
36787,25625,1360,251,23,287,270,103,14,15,3,...,7,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
36788,25362,5679,1347,82,1101,661,575,73,28,11,...,47,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
36789,25262,1242,172,10,158,375,90,8,7,5,...,22,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
36790,25204,314,58,1,98,66,16,3,1,0,...,1,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
